# TextSketch
Este _notebook_ implementa a DSL TextSketch, como descrita no [README](https://github.com/gui-clone/TextSketch).

Um **ponto** (`point`) pode ser descrito num espaço bidimensional. Apesar de ser definido aqui, ele é utilizado apenas para a implementação interna, não fazendo parte da interface da DSL.

In [2]:
(define-syntax point
  (syntax-rules ()
    ((point (a b))
     (format #f "~A ~A" a b))))

A **curva de bézier** (`bezier`) é o principal primitivo da nossa DSL. Com três pontos — inicial, final e de controle, criamos uma curva quadrática. Com a composição dessas curvas, podemos implementar tanto segmentos de reta quanto outras curvas arbitrárias.

Ela é implementada utilizando a diretiva `path` do SVG.

In [3]:
(define-syntax bezier
  (syntax-rules ()
    ((bezier sPT ePT cPT)
     `(path (d . ((,(point sPT) ,(point ePT) ,(string-append "Q " (point cPT)))))
            (stroke . "black")
            (fill . "none")
            (stroke-width . "2")
            (transform . ((translate . "") (scale . "") (rotate . "")))))))

Também podemos implementar uma curva de bézier **quadrática** (`bezier3`), que utiliza dois pontos de controle.

In [17]:
(define-syntax bezier3
  (syntax-rules ()
    ((bezier sPT ePT cPT1 cPT2)
     `(path (d . ((,(point sPT) ,(point ePT) ,(string-append "C " (point cPT1) " " (point cPT2)))))
            (stroke . "black")
            (fill . "none")
            (stroke-width . "2")
            (transform . ((translate . "") (scale . "") (rotate . "")))))))

Uma **forma** (`form`) compõe diversas curvas de bézier numa única diretiva `path` — por isso, seu valor de retorno tem o mesmo tipo de `bezier`.

In [5]:
(define-syntax form
  (syntax-rules ()
    ((form shape ...) ; lista em define-syntax
     (let* ((get_d (lambda (curve) (cadr (assq 'd (cdr curve)))))
            (dList (map get_d (list shape ...)))
            (sPT (caar dList))
            (ePT (cadr (car (list-tail dList (- (length dList) 1)))))
            ;; get the second and third element of every list
            (bodyList (apply append (map (lambda (path) (reverse (cdr path))) dList)))
            ;; remove the last elemet and concatenate strings
            (body (string-join (reverse (cdr (reverse bodyList))) " ")))
       `(path (d . ((,sPT ,ePT ,body)))
              (stroke . "black")
              (fill . "none")
              (stroke-width . "2")
              (transform . ((translate . "") (scale . "") (rotate . ""))))))))

Uma **união** (`union`) compõe diversos elementos num único `group` do SVG, permitindo a composição de diferentes formas numa única unidade.

In [6]:
(define-syntax union
  (syntax-rules ()
    ((union shape ...)
     (let* ((get_shape (lambda (x)
                         (let ((type (car x))
                               (rest (cdr x)))
                           (if (eq? 'group type) (car rest)
                               (cons type rest)))))
            (temp_s (map get_shape (list shape ...)))
            (unite (lambda (x)
                     (if (and (list? x)
                              (list? (car x))
                              (eq? 'path (caar temp_s)))
                         x (list x))))
            (shapes (apply append (map unite temp_s) ) ))
       `(group (,@shapes))))))

O macro `fill` adiciona uma cor de preenchimento à uma forma, sem modificar nenhuma outra propriedade; retornando a nova forma.

In [7]:
(define-syntax fill
  (syntax-rules ()
    ((fill color shape)
     (let ((newD (cdr (assq 'd (cdr shape))))
           (newStroke (cdr (assq 'stroke (cdr shape))))
           (newColor (format #f "~A" (quote color)))
           (newStrokeW (cdr (assq 'stroke-width (cdr shape))))
           (newTransform (cdr (assq 'transform (cdr shape)))))
       `(path (d . ,newD)
              (stroke . ,newStroke)
              (fill . ,newColor)
              (stroke-width . ,newStrokeW)
              (transform . ,newTransform))))))

O macro `stroke` adiciona cor à "borda" do objeto contido.

In [8]:
(define-syntax stroke
  (syntax-rules ()
    ((stroke color shape)
     (let ((newD (cdr (assq 'd (cdr shape))))
           (newStroke (format #f "~A" (quote color)))
           (newColor  (cdr (assq 'fill (cdr shape))))
           (newStrokeW (cdr (assq 'stroke-width (cdr shape))))
           (newTransform (cdr (assq 'transform (cdr shape)))))
       `(path (d . ,newD)
              (stroke . ,newStroke)
              (fill . ,newColor)
              (stroke-width . ,newStrokeW)
              (transform . ,newTransform))))))

O macro `stroke-width` altera a grossura da "borda" do objeto contido.

In [9]:
(define-syntax stroke-width
  (syntax-rules ()
    ((stroke width shape)
     (let ((newD (cdr (assq 'd (cdr shape))))
           (newStroke (cdr (assq 'stroke (cdr shape))))
           (newColor  (cdr (assq 'fill (cdr shape))))
           (newStrokeW (format #f "~A" (quote width)))
           (newTransform (cdr (assq 'transform (cdr shape)))))
       `(path (d . ,newD)
              (stroke . ,newStroke)
              (fill . ,newColor)
              (stroke-width . ,newStrokeW)
              (transform . ,newTransform))))))

O macro `translate` modifica a posição do objeto contido.

In [10]:
(define-syntax translate
  (syntax-rules ()
    ((translate pt shape)
     (let* ((newD (cdr (assq 'd (cdr shape))))
            (newStroke (cdr (assq 'stroke (cdr shape))))
            (newColor  (cdr (assq 'fill (cdr shape))))
            (newStrokeW (cdr (assq 'stroke-width (cdr shape))))
            (newTransform (cdr (assq 'transform (cdr shape))))
            (newTranslate (format #f "translate(~A)" (point pt)))
            (newScale (cdr (assq 'scale newTransform)))
            (newRotate (cdr (assq 'rotate newTransform))))
       `(path (d . ,newD)
              (stroke . ,newStroke)
              (fill . ,newColor)
              (stroke-width . ,newStrokeW)
              (transform . ((translate . ,newTranslate)
                            (scale . ,newScale)
                            (rotate . ,newRotate))))))))

O macro `escale` altera o tamanho do objeto contido, podendo distorcê-lo.

In [11]:
(define-syntax scale
  (syntax-rules ()
    ((scale xScale yScale shape)
      (let* ((newD (cdr (assq 'd (cdr shape))))
             (newStroke (cdr (assq 'stroke (cdr shape))))
             (newColor  (cdr (assq 'fill (cdr shape))))
             (newStrokeW (cdr (assq 'stroke-width (cdr shape))))
             (newTransform (cdr (assq 'transform (cdr shape))))
             (newTranslate (cdr (assq 'translate newTransform)))
             (newScale (format #f "scale(~A ~A)" xScale yScale))
             (newRotate (cdr (assq 'rotate newTransform))))
        `(path (d . ,newD)
           (stroke . ,newStroke)
           (fill . ,newColor)
           (stroke-width . ,newStrokeW)
           (transform . ((translate . ,newTranslate)
                         (scale . ,newScale)
                         (rotate . ,newRotate))))))))

O macro `rotate` rotaciona o objeto contido.

In [12]:
(define-syntax rotate
  (syntax-rules ()
    ((rotate degree shape)
      (let* ((newD (cdr (assq 'd (cdr shape))))
             (newStroke (cdr (assq 'stroke (cdr shape))))
             (newColor  (cdr (assq 'fill (cdr shape))))
             (newStrokeW (cdr (assq 'stroke-width (cdr shape))))
             (newTransform (cdr (assq 'transform (cdr shape))))
             (newTranslate (cdr (assq 'translate newTransform)))
             (newScale (cdr (assq 'scale newTransform)))
             (newRotate (format #f "rotate(~A)" degree)))
        `(path (d . ,newD)
           (stroke . ,newStroke)
           (fill . ,newColor)
           (stroke-width . ,newStrokeW)
           (transform . ((translate . ,newTranslate)
                         (scale . ,newScale)
                         (rotate . ,newRotate))))))))

O macro `new-panel` retorna uma função que pode ser utilizada para converter elementos em uma estrutura SVG, num plano com a largura e altura indicados.

In [13]:
(define-syntax new-panel
  (syntax-rules ()
    ((new-panel width height)
     (lambda (shape)
       (let* ((type (car shape))
              (content (cdr shape))
              (toSVG (lambda (cont)
                       (let* ((newD (cadr (assq 'd cont)))
                              (sPT (car newD))
                              (ePT (cadr newD))
                              (otherD (caddr newD))
                              (newStroke (cdr (assq 'stroke cont)))
                              (newColor (cdr (assq 'fill cont)))
                              (close-str (if (string=? sPT ePT) " Z" ""))
                              (newStrokeW (cdr (assq 'stroke-width cont)))
                              (newTransform (string-join (map cdr (cdr (assq 'transform cont))) " ")))
                         `(<path
                           d= ,(string-append "M " sPT " " otherD " " ePT close-str)
                           stroke= ,(format #f "~A" newStroke)
                           fill= ,(format #f "~A" newColor)
                           stroke-width= ,(format #f "~A" newStrokeW)
                           transform= ,(format #f "~A" newTransform)
                           />)))))
         (if (eq? 'path type)
             (append
              `(<svg
                width="400"
                height="400"
                viewBox= ,(format #f "0 0 ~A ~A" width height) >)
              (toSVG content)
              '(</svg>))
           (append
            `(<svg
              width="400"
              height="400"
              viewBox= ,(format #f "0 0 ~A ~A" width height) >)
            (map (lambda (x) (toSVG (cdr x))) (car content))
            '(</svg>))))))))

O macro `defineSVG` possibilita a construção de formas/grupos com diferentes argumentos, possibilitando a criação automatizada de diferentes variações de um mesmo objeto. Ele retorna outro macro, que pode ser chamado com os argumentos correspondentes para produzir a forma final.

In [7]:
(define-syntax defineSVG
  (syntax-rules ()
    ((defineSVG name (args ...)
       template)
     (define-syntax name
       (syntax-rules ()
         ((name args ...) template))))))